In [7]:
from datetime import datetime
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv('ibv_input_data.csv')
df = df.dropna(subset=['MaxHistory'])

In [9]:
df.head()

,AppID,CustomerID,ApplicationDate,Age,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,INCOME_SOURCES_COUNT,MONTH_INFLOW_MEAN,MONTH_INFLOW_SD,MONTH_OUTFLOW_MEAN,MONTH_OUTFLOW_SD,MONTHS_WITH_FEES_RATE,MONTHS_WITH_EMPLOYMENT_RATE,NO_ACTIVITY_RATE,OD_AND_NSF_FEES_DAILY,RECURRENT_COUNT,RECURRING_RATE,BALANCE_MEAN,BALANCE_SD,BALANCE_ABOVE_RATE,LAST_BALANCE_ABOVE,AVG_BALANCE_ABOVE_7D,HIGHER_BALANCE_7D,NUM_LOAN_PMT,TOTAL_LOAN_PMT_AMT,NUM_LOAN_ORIG,TOTAL_LOAN_ORIG_AMT,PAYROLL_AMOUNT,TOTAL_CREDIT,TOTAL_DEBIT,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,IsGood,Payin,FPDAA,FPDFA
0,276,217,2023-05-03 12:56:45,28.0,89,0.991452,0.015796,265.159083,264.257750,178.074111,-1.496047,-1.471057,5342.223333,4.00,0.0,7927.7325,5028.665391,7954.7725,5046.131630,0.75,0.0,0.955056,4.550562,0.0,0.0,32.239780,44.617901,0.000000,0,0,0,0.0,0.00,0.0,0.0,16026.67,31710.93,31819.09,0,9,0,0,0,0.0,658.0,31710.93,657.0,0,0.268933,1.0,1.0
1,559,463,2023-05-10 11:09:47,29.0,23,1.079348,0.350087,114.751833,126.027333,63.722167,0.707107,0.707107,1911.665000,2.00,0.0,3780.8200,660.833713,3592.5550,552.992858,0.50,0.0,0.695652,0.000000,0.0,0.0,216.366889,440.634449,0.222222,1,1,1,2.0,300.00,0.0,0.0,3822.93,7561.64,7185.11,0,0,0,0,0,0.0,68.0,7561.64,60.0,0,0.830400,0.0,0.0
2,861,709,2023-05-15 14:02:43,46.0,90,0.995773,0.046297,240.609250,236.780583,92.236417,-0.726860,-0.314176,3369.577500,21.00,0.0,7228.4175,4726.787789,7251.1525,4826.488975,1.00,0.0,0.555556,1.205444,0.0,0.0,65.120220,250.639644,0.098901,0,0,0,1.0,131.50,1.0,500.0,11063.15,28913.67,29004.61,0,14,3,1,0,0.0,674.0,28913.67,614.0,0,0.270900,1.0,1.0
3,1044,833,2023-05-17 15:14:30,57.0,83,1.188739,0.370218,150.916667,172.160500,56.491444,-0.244696,-0.115385,3818.757500,8.75,0.0,5289.8150,1738.630021,4823.6800,2030.802974,0.75,0.0,0.650602,0.000000,0.0,0.0,831.042198,535.658941,0.824176,1,1,1,6.0,1184.72,1.0,500.0,5080.23,21159.26,19294.72,0,0,0,0,14,358.0,219.0,21159.26,158.0,1,1.607700,0.0,0.0
4,1380,1014,2023-05-23 12:56:18,28.0,90,1.053158,0.093006,410.537917,438.818750,0.776000,-0.099578,0.046084,7580.980000,24.75,0.0,13439.5625,7119.594692,12667.4525,6513.799077,1.00,0.0,0.666667,2.180111,0.0,0.0,646.449890,779.309063,0.659341,1,1,1,6.0,1405.26,2.0,1100.0,0.00,53758.25,50669.81,0,2,4,5,172,6788.0,791.0,53758.25,755.0,0,0.000000,1.0,1.0


In [10]:
len(df)

2807

In [11]:
df.dtypes

AppID                            int64
CustomerID                       int64
ApplicationDate                 object
Age                            float64
MaxHistory                       int64
CREDIT_TO_DEBIT_RATIO_MEAN     float64
CREDIT_TO_DEBIT_RATIO_SD       float64
DAILY_DEBIT_AMOUNT_MEAN        float64
DAILY_INCOME_MEAN              float64
DAILY_INCOME_REGULAR_MEAN      float64
DEBIT_AMOUNT_Z                 float64
DEBIT_COUNT_Z                  float64
HIGHEST_PAY_DEPOSIT_MEAN       float64
HIGHEST_PAY_FREQUENCY          float64
INCOME_SOURCES_COUNT           float64
MONTH_INFLOW_MEAN              float64
MONTH_INFLOW_SD                float64
MONTH_OUTFLOW_MEAN             float64
MONTH_OUTFLOW_SD               float64
MONTHS_WITH_FEES_RATE          float64
MONTHS_WITH_EMPLOYMENT_RATE    float64
NO_ACTIVITY_RATE               float64
OD_AND_NSF_FEES_DAILY          float64
RECURRENT_COUNT                float64
RECURRING_RATE                 float64
BALANCE_MEAN             

# Model Training

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_curve, auc

def score(val, pred):
    fpr, tpr, thresholds = roc_curve(val, pred[:,1]) 
    roc_auc = auc(fpr, tpr) 
    return roc_auc

features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT', 'NUM_GAMBLING_PMT', 'AMT_GAMBLING_PMT', 'NUM_EI_PMT', 'AMT_EI_PMT',
                   'NUM_Bree']

x = df[df.Age>35]
y = df[df.Age>35][['IsGood']]
x[features_modeling].to_csv('trainingforstandardization.csv' ,index=False)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 1/4, shuffle = False, stratify = None)
print(len(df))
scaler = StandardScaler()
scaler.fit(x_train[features_modeling])
x_train[features_modeling] = scaler.transform(x_train[features_modeling])
x_val[features_modeling] = scaler.transform(x_val[features_modeling])
x[features_modeling] = scaler.transform(x[features_modeling])

2807


In [13]:
def model_evaluation(model):
    clf = model
    clf.fit(x_train[features_modeling], y_train)
    pred_dt = clf.predict_proba(x_val[features_modeling])
    pred1_dt = clf.predict_proba(x_train[features_modeling])
    print(score(y_train, pred1_dt), score(y_val, pred_dt)) 
    print(confusion_matrix(y_train.values, pred1_dt[:,1]>0.37))
    print(confusion_matrix(y_val.values, pred_dt[:,1]>0.37))
    
    def get_score1(x, y, Kfold, num_of_folds=4):
        def score(val, pred):
            fpr, tpr, thresholds = roc_curve(val, pred[:,1]) 
            roc_auc = auc(fpr, tpr) 
            return roc_auc
        sum_of_score = 0
        print('model:',model)
        for train, test in Kfold.split(x, y):
            x_t = x[features_modeling].iloc[train,:]
            y_t = y.iloc[train,:]
            x_v = x[features_modeling].iloc[test,:]
            y_v = y.iloc[test,:]
            clf = model
            clf.fit(x_t, np.array(y_t).ravel())
            pred = clf.predict_proba(x_v)
            print('kfold: ',score(y_v, pred))
            sum_of_score += score(y_v, pred)
        return sum_of_score / num_of_folds
    Kfold = StratifiedKFold(n_splits = 5, shuffle = False)
    rfmd5 = get_score1(x, y, Kfold, num_of_folds=5)
    return rfmd5

In [14]:
def model_application_evaluation(df_, model, training = True, kmeans = 0):
    df = df_.copy()
    if training:
        df[features_modeling] = scaler.transform(df[features_modeling])
    pred = model.predict_proba(df[df.Age>35][features_modeling])[:,1]
    num_bands = 5

    pred2 = pd.DataFrame()
    pred2['Age'] = df[df.Age>35].Age
    pred2['NBScore'] = 1000 * pred
    s_score =  pred2['NBScore'].values.reshape(-1, 1)
    if training:
        kmeans = KMeans(n_clusters=num_bands, random_state=0)
        kmeans.fit(s_score)
    pred2.loc[:, 'ClusterID'] = kmeans.predict(s_score)
    band_boundary = pred2.groupby(f'ClusterID')['NBScore'].agg(['min', 'max']).sort_values(by='min')
    band_boundary['ColBand'] = list(range(1, len(band_boundary)+1, 1))
    pred2['NBBand'] = pred2['ClusterID'].map(band_boundary[['ColBand']].to_dict()['ColBand'])
    pred2['IsGood'] = df[df.Age>35]['IsGood'].values
    pred2['Payin'] = df[df.Age>35]['Payin'].values
    k = pred2.groupby('NBBand').agg({'ClusterID':'count','IsGood':['sum','mean'],'Payin':'mean'})
    k['GoodPayin'] = pred2[pred2['Payin'] > 1].groupby('NBBand')['Payin'].mean()
    k['LowPayin'] = pred2[pred2['Payin'] < 1].groupby('NBBand')['Payin'].mean()
    k.columns = ['# Loans','# IsGood', 'IsGood Rate (%)','AvgPayin','GoodPayin','LowPayin']
    k['IsGood Rate (%)'] *= 100
    k['Count Pct (%)'] = k['# Loans'] / np.sum(k['# Loans'])* 100
#     print(k.columns)
    k['Cumulative Payin ⬆️'] = [ np.sum(k.iloc[i:,3] * k.iloc[i:,0])/np.sum(k.iloc[i:,0]) for i in range(len(k))]
    if training:
        print(band_boundary)
        return kmeans, k[['AvgPayin','GoodPayin','LowPayin','# Loans', 'Count Pct (%)', '# IsGood', 'IsGood Rate (%)', 'Cumulative Payin ⬆️']]
    else: 
        return k[['AvgPayin','GoodPayin','LowPayin','# Loans', 'Count Pct (%)', '# IsGood', 'IsGood Rate (%)', 'Cumulative Payin ⬆️']]

### Logistic Regression

In [15]:
clf = LogisticRegression(penalty='l2')
print('model_evaluation',model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
print('Testing Result: \n')
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.6938969217655746 0.6967258448094684
[[115 579]
 [ 36 887]]
[[ 31 221]
 [ 13 274]]
model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
kfold:  0.6386472379295345
kfold:  0.6747562202107656
kfold:  0.6377847741484106
kfold:  0.7001399274126546
kfold:  0.7013205649569287
model_evaluation 0.6705297449316588
                  min         max  ColBand
ClusterID                                 
4            1.750246  360.400399        1
2          361.437727  490.875559        2
0          491.222500  613.206391        3
3          613.598214  756.368250        4
1          758.672003  998.534529        5
Training Result: 
         AvgPayin  GoodPayin  LowPayin  # Loans  Count Pct (%)  # IsGood  IsGood Rate (%)  Cu

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.574671,1.398060,0.217869,43,7.977737,13,30.232558,0.914085
2,0.676457,1.464066,0.236323,161,29.870130,57,35.403727,0.943510
3,0.965333,1.534759,0.328475,161,29.870130,85,52.795031,1.071854
4,1.072182,1.445418,0.315890,115,21.335807,76,66.086957,1.170417
5,1.361891,1.404855,0.137416,59,10.946197,56,94.915254,1.361891


In [16]:
clf = LogisticRegression(penalty='l1',solver='liblinear',random_state=122)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.692836915083942 0.6937531110004977
[[104 590]
 [ 32 891]]
[[ 27 225]
 [ 12 275]]
model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=122, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
kfold:  0.6422357546759461
kfold:  0.6719139446412175
kfold:  0.6382220473129563
kfold:  0.6980847435392891
kfold:  0.6996151996151997
0.6700143379569217
                  min         max  ColBand
ClusterID                                 
1            2.205988  384.495415        1
3          385.259251  508.820066        2
0          509.069115  627.023364        3
4          627.179425  765.900652        4
2          766.144270  997.448641        5
Training Result: 
         AvgPayin  GoodPayin  LowPayin  # Loans  Count Pct (%)  # IsGood  IsGood Rate (%)  Cumulative Payin 

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.541159,1.405134,0.182926,59,10.946197,17,28.813559,0.913952
2,0.739995,1.477384,0.253319,167,30.983302,66,39.520958,0.959774
3,1.005653,1.519438,0.352413,159,29.499072,89,55.974843,1.077036
4,1.041669,1.447057,0.288806,100,18.552876,64,64.000000,1.150737
5,1.352714,1.399456,0.137416,54,10.018553,51,94.444444,1.352714


### Random Forest

In [17]:
clf = RandomForestClassifier(max_depth=5, random_state=1235)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.8273141397710136 0.73249543719927
[[ 99 595]
 [  4 919]]
[[ 26 226]
 [  5 282]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1235,
                       verbose=0, warm_start=False)
kfold:  0.6574815137016093
kfold:  0.6674100310463947
kfold:  0.6573746119200665
kfold:  0.7058463422099786
kfold:  0.7368708732345097
0.6849966744225118
                  min         max  ColBand
ClusterID                                 
3          231.579702  396.443487        1
1          396.947404  494.987041        2
4          495.014514  584.291029        

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.355548,1.364720,0.183251,48,8.905380,7,14.583333,0.914285
2,0.730549,1.565398,0.287569,152,28.200371,52,34.210526,0.968907
3,0.919925,1.448870,0.295476,157,29.128015,85,54.140127,1.075782
4,1.161233,1.482266,0.358652,77,14.285714,55,71.428571,1.210230
5,1.246161,1.420252,0.201610,105,19.480519,88,83.809524,1.246161


In [18]:
clf = RandomForestClassifier(max_depth=4, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.770788776105982 0.7159725678889443
[[ 71 623]
 [  5 918]]
[[ 23 229]
 [  2 285]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
kfold:  0.6565680730752501
kfold:  0.6621190257553895
kfold:  0.6518649700467882
kfold:  0.6903231448685994
kfold:  0.7348594166775984
0.679146926084725
                  min         max  ColBand
ClusterID                                 
3          275.433175  413.460704        1
1          413.741489  499.886431        2
4          500.034317  587.419547        3

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.436518,1.399759,0.204626,67,12.430427,13,19.402985,0.914377
2,0.755100,1.516204,0.304076,131,24.304267,48,36.641221,0.982208
3,0.957858,1.484939,0.297248,169,31.354360,94,55.621302,1.069455
4,1.047692,1.454370,0.265619,76,14.100186,49,64.473684,1.179106
5,1.283142,1.430685,0.250339,96,17.810761,83,86.458333,1.283142


In [19]:
clf = RandomForestClassifier(max_depth=3, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.730705224474758 0.7148940877163874
[[ 28 666]
 [  2 921]]
[[  5 247]
 [  0 287]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
kfold:  0.6518486298390604
kfold:  0.6696401241855787
kfold:  0.647251738160829
kfold:  0.692815601906511
kfold:  0.7314705496523678
0.6786053287488695
                  min         max  ColBand
ClusterID                                 
2          328.749406  437.151493        1
0          437.436501  509.448407        2
4          509.600288  591.249056        3


,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.461630,1.409679,0.190759,72,13.358071,16,22.222222,0.914531
2,0.796852,1.513532,0.293471,145,26.901670,59,40.689655,0.984358
3,0.953479,1.479588,0.333680,159,29.499072,86,54.088050,1.068793
4,0.990710,1.431062,0.262436,69,12.801484,42,60.869565,1.181278
5,1.321163,1.444959,0.151975,94,17.439703,84,89.361702,1.321163


In [21]:
clf = RandomForestClassifier(max_depth=2, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.6950638657928507 0.7036668325866933
[[  0 694]
 [  0 923]]
[[  0 252]
 [  0 287]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
kfold:  0.6500217485863419
kfold:  0.6524115615024706
kfold:  0.644092439546985
kfold:  0.6868468232104596
kfold:  0.7166032620578074
0.669995166980813
                  min         max  ColBand
ClusterID                                 
4          378.072523  461.517185        1
1          462.519497  519.319313        2
0          519.571714  583.268963        3

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.515977,1.477018,0.201153,77,14.285714,19,24.675325,0.914560
2,0.804423,1.449810,0.308692,124,23.005566,53,42.741935,0.980990
3,0.921357,1.511365,0.310768,175,32.467532,89,50.857143,1.045766
4,0.969195,1.430622,0.238603,62,11.502783,37,59.677419,1.179334
5,1.308330,1.440852,0.224059,101,18.738404,89,88.118812,1.308330


In [22]:
clf = RandomForestClassifier(max_depth=1, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.6691428776605544 0.6884160168132293
[[  0 694]
 [  0 923]]
[[  0 252]
 [  0 287]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=1, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
kfold:  0.6395171813832101
kfold:  0.6466723512178058
kfold:  0.6376973195155013
kfold:  0.6738598102234465
kfold:  0.7031243167606805
0.6601741958201289
                  min         max  ColBand
ClusterID                                 
2          461.779774  501.326696        1
0          501.498157  534.579329        2
4          534.611735  574.595194       

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.595780,1.439737,0.254952,73,13.543599,21,28.767123,0.914670
2,0.792551,1.477155,0.262534,111,20.593692,48,43.243243,0.964625
3,0.875938,1.501132,0.308160,188,34.879406,89,47.340426,1.018429
4,1.008868,1.396525,0.218646,79,14.656772,51,64.556962,1.178837
5,1.331423,1.470520,0.246465,88,16.326531,78,88.636364,1.331423


In [20]:
clf = RandomForestClassifier(max_depth=3, random_state=123)
print(model_evaluation(clf))
importances = clf.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
feat_labels = x[features_modeling].columns

for f in range(x.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 20,
                            feat_labels[sorted_indices[f]],
                            importances[sorted_indices[f]]))

0.730705224474758 0.7148940877163874
[[ 28 666]
 [  2 921]]
[[  5 247]
 [  0 287]]
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
kfold:  0.6518486298390604
kfold:  0.6696401241855787
kfold:  0.647251738160829
kfold:  0.692815601906511
kfold:  0.7314705496523678
0.6786053287488695
 1) DAILY_INCOME_REGULAR_MEAN 0.141171
 2) PAYROLL_AMOUNT       0.109057
 3) BALANCE_ABOVE_RATE   0.087433
 4) NUM_LOAN_ORIG        0.067022
 5) BALANCE_SD           0.063376
 6) BALANCE_MEAN         0.056773
 7) TO

IndexError: index 43 is out of bounds for axis 0 with size 43

### XGBoost

In [23]:
clf =  xgb.XGBClassifier(max_depth=3, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.9054205525772681 0.7300481168076987
[[264 430]
 [  9 914]]
[[ 54 198]
 [ 21 266]]
model: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
kfold:  0.6391692040017399
kfold:  0.6520180156543793
kfold:  0.6609383882111155
kfold:  0.7138265774629411
kfold:  0.7359744632471906
0.6803853297154733
                  min         max  ColBand
ClusterID                                 
2           72.698082  326.607300        1
0          328.200226  471.571808        2
4          472.708313  607.795105        3
3          608.846252  750.797424        4
1          751.890625  95

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.504038,1.552502,0.204476,63,11.688312,14,22.222222,0.914392
2,0.618441,1.481831,0.274941,131,24.304267,37,28.244275,0.968704
3,0.900125,1.456054,0.292879,137,25.417440,71,51.824818,1.101702
4,1.243995,1.512705,0.318438,120,22.263451,93,77.500000,1.234472
5,1.221485,1.389605,0.332848,88,16.326531,72,81.818182,1.221485


In [24]:
clf =  xgb.XGBClassifier(max_depth=2, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.8099824841311223 0.7313616503511973
[[169 525]
 [ 17 906]]
[[ 36 216]
 [ 17 270]]
model: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
kfold:  0.6483471074380165
kfold:  0.6608946608946609
kfold:  0.6622502077047532
kfold:  0.7168437622983077
kfold:  0.7452009270191089
0.6867073330709694
                  min         max  ColBand
ClusterID                                 
3          103.942520  361.098511        1
0          362.719757  491.941345        2
4          492.037415  609.637146        3
1          610.479553  728.960388        4
2          730.406982  91

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.501898,1.536680,0.202961,58,10.760668,13,22.413793,0.913979
2,0.647999,1.507658,0.267765,152,28.200371,46,30.263158,0.963669
3,0.949234,1.464365,0.301166,140,25.974026,78,55.714286,1.109510
4,1.156514,1.487800,0.280973,102,18.923933,74,72.549020,1.228233
5,1.312318,1.407798,0.484823,87,16.141002,76,87.356322,1.312318


In [25]:
clf =  xgb.XGBClassifier(max_depth=1, random_state=123)
print(model_evaluation(clf))
a,training_result = model_application_evaluation(df, clf)
print('Training Result: \n',training_result)
testing_result = model_application_evaluation(x_val, clf, training = False, kmeans = a)
testing_result

0.7192106306649473 0.7149148277197057
[[ 81 613]
 [ 22 901]]
[[ 18 234]
 [  9 278]]
model: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
kfold:  0.653284036537625
kfold:  0.6511544011544012
kfold:  0.6534719489264944
kfold:  0.6998884953430409
kfold:  0.7247365429183611
0.6765070849759844
                  min         max  ColBand
ClusterID                                 
1          206.869629  397.925629        1
3          398.315918  493.001526        2
2          493.394989  587.707764        3
4          588.276489  684.957458        4
0          685.083374  857

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,0.611896,1.539657,0.214285,50,9.276438,15,30.000000,0.914478
2,0.697805,1.479589,0.284449,134,24.860853,46,34.328358,0.945417
3,0.867014,1.482334,0.266702,163,30.241187,80,49.079755,1.038881
4,1.112020,1.476939,0.346867,96,17.810761,65,67.708333,1.184789
5,1.257559,1.418209,0.231870,96,17.810761,81,84.375000,1.257559


### Save file for training model

In [26]:
clf = RandomForestClassifier(max_depth=2, random_state=123) 
# --> knock out 43% of vol with 49% FPD and drops FPD from 34% tp 23%
clf.fit(x[features_modeling],y)
joblib.dump(clf, "IBV_Model_V3_1_RFD2.pkl")

clf = LogisticRegression(penalty='l1',solver='liblinear',random_state=123)
# --> knock out 55% of vol with 45% FPD and drops FPD from 34% tp 20%
clf.fit(x[features_modeling],y)
joblib.dump(clf, "IBV_Model_V3_2_LGL1.pkl")

['IBV_Model_V3_2_LGL1.pkl']

In [27]:
x.to_csv('traininginputdata.csv', index = False)
y.to_csv('trainingoutputdata.csv', index = False)

In [28]:
x

,AppID,CustomerID,ApplicationDate,Age,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,INCOME_SOURCES_COUNT,MONTH_INFLOW_MEAN,MONTH_INFLOW_SD,MONTH_OUTFLOW_MEAN,MONTH_OUTFLOW_SD,MONTHS_WITH_FEES_RATE,MONTHS_WITH_EMPLOYMENT_RATE,NO_ACTIVITY_RATE,OD_AND_NSF_FEES_DAILY,RECURRENT_COUNT,RECURRING_RATE,BALANCE_MEAN,BALANCE_SD,BALANCE_ABOVE_RATE,LAST_BALANCE_ABOVE,AVG_BALANCE_ABOVE_7D,HIGHER_BALANCE_7D,NUM_LOAN_PMT,TOTAL_LOAN_PMT_AMT,NUM_LOAN_ORIG,TOTAL_LOAN_ORIG_AMT,PAYROLL_AMOUNT,TOTAL_CREDIT,TOTAL_DEBIT,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,IsGood,Payin
0,17011,8857,2024-03-20 13:18:11,36.0,-0.006978,-0.035121,-0.025257,2.439592,2.252466,0.152316,-0.263406,-0.540458,2.548283,0.043612,0.0,2.142196,4.332706,2.252587,3.995754,-1.214259,0.0,-0.368886,-0.353323,-0.10968,-0.107126,0.681165,1.223027,1.093603,-0.577804,1.196940,-0.766104,-0.468682,-0.544750,-0.414399,-0.436897,0.018152,2.106230,2.216503,-0.156668,-0.366071,-0.554249,-0.378391,0.936544,8.354895,0,0.000000
1,17111,8909,2024-03-21 14:10:48,41.0,-0.006978,-0.033669,-0.025719,1.220148,1.554605,2.305558,0.277665,0.265079,0.699292,0.335677,0.0,1.596941,1.155124,1.544754,1.274823,0.946241,0.0,0.488627,0.323533,-0.10968,-0.107126,-0.268391,0.037153,-0.449869,-0.577804,-0.835464,-0.766104,4.103604,2.690913,-0.023265,0.485966,2.620285,1.574849,1.525734,-0.156668,0.029140,0.895851,0.706620,-0.091044,0.041857,1,2.032666
2,17320,9000,2024-03-26 11:56:37,38.0,-0.030865,-0.035498,-0.023596,-0.407202,-0.326741,-0.452004,0.140466,0.058612,0.157946,-0.235472,0.0,-0.372037,-0.044457,-0.370814,0.011031,0.226075,0.0,0.463226,-0.347180,-0.10968,-0.107126,-0.128376,0.120841,-0.144419,1.730692,-0.835464,-0.766104,0.837685,0.159509,-0.179719,-0.328205,-0.469250,-0.344030,-0.343655,-0.156668,0.503394,0.170801,1.610796,-0.221119,-0.159332,0,0.549666
3,17379,9024,2024-03-27 09:13:55,43.0,-0.030865,-0.034430,-0.025671,1.046162,1.067161,1.244477,1.081029,0.631907,0.378238,-0.585951,0.0,0.981066,1.057235,0.942805,1.024342,0.946241,0.0,0.069068,0.761698,-0.10968,-0.107126,0.565793,0.776978,0.656981,1.730692,-0.835464,-0.766104,-0.387035,-0.488891,-0.414399,-0.436897,0.899013,0.974644,0.938297,-0.156668,0.187225,0.412484,1.610796,-0.208111,-0.156260,1,1.354500
4,17954,9318,2024-04-05 08:31:12,55.0,-0.054752,-0.034957,-0.024508,0.417524,0.880947,1.201147,-1.177135,-1.242490,0.874554,-0.248453,0.0,1.111199,0.980571,1.092516,0.581993,0.226075,0.0,0.756157,-0.353323,-0.10968,-0.107126,0.782439,0.365061,2.344139,1.730692,1.196940,1.305305,6.389748,5.199273,-0.023265,1.645486,0.859168,1.101466,1.084398,-0.156668,-0.366071,-0.554249,-0.378391,-0.221119,-0.159332,1,1.607700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,25309,15520,2024-07-29 10:21:25,37.0,1.927868,-0.033537,-0.025860,0.525706,0.532881,-0.920406,0.061842,1.080400,0.706799,3.733370,0.0,0.592378,-0.371549,0.489949,-0.438929,-0.494092,0.0,0.492730,-0.353323,-0.10968,-0.107126,0.291707,0.213727,1.135318,-0.577804,1.196940,1.305305,-0.142091,0.195175,1.541270,0.919917,-0.847029,1.474503,1.319820,-0.156668,-0.366071,-0.554249,-0.378391,-0.195104,-0.159332,1,1.619500
2798,25441,15617,2024-07-29 14:48:46,46.0,-0.006978,-0.033793,-0.025526,0.355373,0.120469,-0.920406,0.739433,1.669680,-0.552846,1.289757,0.0,0.324823,0.175113,0.280730,0.119614,0.946241,0.0,0.644539,-0.235997,-0.10968,-0.107126,0.438018,0.314373,1.260464,-0.577804,-0.835464,-0.766104,-0.305387,-0.411985,1.228363,1.343393,-0.847029,0.335098,0.292183,-0.156668,-0.207986,-0.554249,-0.016720,-0.221119,-0.159332,1,1.604750
2799,25559,15694,2024-07-30 11:53:01,55.0,-0.006978,-0.033707,-0.024876,-0.100435,-0.092518,-0.920406,-0.847921,-0.562090,0.399079,3.60680

In [29]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
import joblib
import json
from sqlalchemy import create_engine 
from datetime import datetime, date
pd.set_option('display.max_columns', None)

In [30]:
jsonfilepath = "creditbooksampleJSON.json"
modelfilepath = "NegativeDB_model_v2.pkl"
datacleanfilepath = 'dataclean_nb_fillna.csv'
%run -i ndmodel.py jsonfilepath modelfilepath datacleanfilepath

Exception: File `'ndmodel.py'` not found.

### Run IBV Model together with ND model

In [24]:
nd_df = pd.read_csv('../../ND Model/V3/traininginputdata.csv')
try: del nd_df['Unnamed: 0'] 
except: pass
clf_nd = joblib.load('../../ND Model/V3/NegativeDB_model_v3.pkl')
nd_df = nd_df[nd_df.Age>35]

features_modeling = ['frequency', 'avg_amountReq', 'med_amountReq',
       'std_amountReq', 'Yeardiff_max', 'Yeardiff_min', 'Yeardiff_mean',
       'Yeardiff_median', 'Monthdiff_max', 'Monthdiff_min', 'Monthdiff_mean',
       'Monthdiff_median', 'Weekdiff_max', 'Weekdiff_min', 'Weekdiff_mean',
       'Weekdiff_median', 'Daydiff_max', 'Daydiff_min', 'Daydiff_mean',
       'Daydiff_median', 'have_valid_phone', 'times_valid_phone',
       'total_phone_enter', 'correct_phone_rate', 'num_unique_valid_phone',
        'Loanspaidoff_count', 'Incollection_count',
       'Loanspaidoff_count_in30days', 'Incollection_count_in30days',
       'Loanspaidoff_rate', 'Fraudster_app_count', 'Fraudster_lender_count',
       'Fraudster_app_count_in30days', 'Fraudster_lender_count_in30days',
       'Refused_count', 'Refused_rate', 'Refused_count_within30days',
       'Refused_rate_within30days', 'Refused_count_before30days',
       'Refused_rate_before30days']

nd_df['NBScore'] = 1000 * clf_nd.predict_proba(nd_df[features_modeling])[:,1]

nd_df['NBBand'] = np.where(nd_df['NBScore']<234, 1, 
                  np.where(nd_df['NBScore']<312, 2,
                  np.where(nd_df['NBScore']<420, 3,
                  np.where(nd_df['NBScore']<624, 4, 5))))

FileNotFoundError: [Errno 2] No such file or directory: '../../ND Model/V3/NegativeDB_model_v3.pkl'

In [132]:
clf_nd

RandomForestClassifier(max_depth=3, random_state=5300)

In [90]:
ibv_df1 = pd.read_csv('../../IBV Model/V3/traininginputdata.csv')
try: del ibv_df1['Unnamed: 0'] 
except: pass
clf_nd = joblib.load('../../IBV Model/V3/IBV_Model_V3_1_RFD2.pkl')
ibv_df1 = ibv_df1[ibv_df1.Age>35]

features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT']

ibv_df1['IBVScore'] = 1000 * clf_nd.predict_proba(ibv_df1[features_modeling])[:,1]

ibv_df1['IBVBand'] = np.where(ibv_df1['IBVScore']<240, 1, 
                  np.where(ibv_df1['IBVScore']<273, 2,
                  np.where(ibv_df1['IBVScore']<310, 3,
                  np.where(ibv_df1['IBVScore']<376, 4, 5))))

In [91]:
ibv_df2 = pd.read_csv('../../IBV Model/V3/traininginputdata.csv')
try: del ibv_df2['Unnamed: 0'] 
except: pass
clf_nd = joblib.load('../../IBV Model/V3/IBV_Model_V3_2_LGL1.pkl')
ibv_df2 = ibv_df2[ibv_df2.Age>35]

features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT']

ibv_df2['IBVScore'] = 1000 * clf_nd.predict_proba(ibv_df2[features_modeling])[:,1]

ibv_df2['IBVBand'] = np.where(ibv_df2['IBVScore']<240, 1, 
                  np.where(ibv_df2['IBVScore']<273, 2,
                  np.where(ibv_df2['IBVScore']<310, 3,
                  np.where(ibv_df2['IBVScore']<376, 4, 5))))

In [112]:
# ND: knock out bank 3 and 4 and 5
# IBV knock out the band 4 and 5
# check the overlapping and impact on the swap out

nd_ibv = nd_df[['FPDAA','ApplicationDate','AppID','Age','NBBand']].merge(ibv_df1[['AppID','IBVBand']], on = 'AppID')
nd_ibv['Knockout'] = 0
nd_ibv['Knockout_ND'] = 0
nd_ibv.loc[nd_ibv.NBBand>=3, ['Knockout','Knockout_ND']] = 1.0
knockout_rate_nd = np.mean(nd_ibv['Knockout'])
FPD_AfterND = np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA)
print('ND Model knockout: ', len(nd_ibv['Knockout']), np.mean(nd_ibv['Knockout']), np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA))

nd_ibv['Knockout_IBV'] = 0
nd_ibv.loc[nd_ibv.IBVBand>=4, ['Knockout','Knockout_IBV']] = 1.0
knockout_rate_ibv = np.mean(nd_ibv['Knockout'])
FPD_AfterIBV = np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA)
print('IBV Model knockout: ', len(nd_ibv['Knockout']), np.mean(nd_ibv['Knockout']), np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA))

print('Additional knock out rate: ', knockout_rate_ibv - knockout_rate_nd)
print('Additional FPD rate drop: ', FPD_AfterND - FPD_AfterIBV)

ND Model knockout:  847 0.269185360094451 0.16478190630048464
IBV Model knockout:  847 0.3990554899645809 0.13163064833005894
Additional knock out rate:  0.1298701298701299
Additional FPD rate drop:  0.0331512579704257


In [115]:
# Count in grid
nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).count().iloc[:,0].unstack()

Knockout_IBV,0,1
Knockout_ND,,
0,509,110
1,142,86


In [122]:
# Percentage in grid
(nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).count().iloc[:,0].unstack())/(0.0+len(nd_ibv))*100

Knockout_IBV,0,1
Knockout_ND,,
0,60.094451,12.987013
1,16.765053,10.153483


In [120]:
# FPD in grid
(nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).mean()['FPDAA'].unstack()*100).round(2)

Knockout_IBV,0,1
Knockout_ND,,
0,13.16,31.82
1,45.07,81.40


In [125]:
# ND: knock out bank 3 and 4 and 5
# IBV knock out the band 4 and 5
# check the overlapping and impact on the swap out

nd_ibv = nd_df[['FPDAA','ApplicationDate','AppID','Age','NBBand']].merge(ibv_df2[['AppID','IBVBand']], on = 'AppID')
nd_ibv['Knockout'] = 0
nd_ibv['Knockout_ND'] = 0
nd_ibv.loc[nd_ibv.NBBand>=3, ['Knockout','Knockout_ND']] = 1.0
knockout_rate_nd = np.mean(nd_ibv['Knockout'])
FPD_AfterND = np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA)
print('ND Model knockout: ', len(nd_ibv['Knockout']), np.mean(nd_ibv['Knockout']), np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA))

nd_ibv['Knockout_IBV'] = 0
nd_ibv.loc[nd_ibv.IBVBand>=4, ['Knockout','Knockout_IBV']] = 1.0
knockout_rate_ibv = np.mean(nd_ibv['Knockout'])
FPD_AfterIBV = np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA)
print('IBV Model knockout: ', len(nd_ibv['Knockout']), np.mean(nd_ibv['Knockout']), np.mean(nd_ibv[nd_ibv['Knockout']==0].FPDAA))

print('Additional knock out rate: ', knockout_rate_ibv - knockout_rate_nd)
print('Additional FPD rate drop: ', FPD_AfterND - FPD_AfterIBV)

ND Model knockout:  847 0.269185360094451 0.16478190630048464
IBV Model knockout:  847 0.46871310507674147 0.11333333333333333
Additional knock out rate:  0.19952774498229048
Additional FPD rate drop:  0.051448572967151315


In [126]:
# Count in grid
nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).count().iloc[:,0].unstack()

Knockout_IBV,0,1
Knockout_ND,,
0,450,169
1,118,110


In [127]:
# Percentage in grid
(nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).count().iloc[:,0].unstack())/(0.0+len(nd_ibv))*100

Knockout_IBV,0,1
Knockout_ND,,
0,53.128689,19.952774
1,13.931523,12.987013


In [129]:
# FPD in grid
(nd_ibv.groupby(['Knockout_ND','Knockout_IBV']).mean()['FPDAA'].unstack()*100).round(2)

Knockout_IBV,0,1
Knockout_ND,,
0,11.33,30.18
1,42.37,76.36


In [97]:
clf.feature_names_in_, len(clf.feature_names_in_)

38